In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import re
from collections import namedtuple
from typing import List, Dict, Union, NamedTuple

import prextract.title_filter as title_filter
import fitz

In [3]:
PATH='2.pdf'
doc = fitz.open(PATH)
pnum = 1

p = doc[pnum]
p_width = p.MediaBox[2]


class TextBlockTrans(NamedTuple):
    x0: float
    y0: float
    x1: float
    y1: float
    text: str
    block_no: int
    page: int
    pwidth: float = None
    def __repr__(self):
        ret = []
        ret.append('TextBlockTrans')
        ret.append('\tbbox: ({}, {}, {}, {})'.format(*self[:4]))
        ret.append('\ttext: {}'.format(self.text))
        ret.append('\tblock_no: {}'.format(self[5]))
        ret.append('\tpage, pwidth: {}'.format(self[-2:]))
        return '\n'.join(ret)


In [4]:
text_blocks = p.getTextBlocks()
extract = p.getTextPage().extractDICT()['blocks']
text_blocks[4:6]

[(56.69187545776367,
  441.5146789550781,
  404.0799865722656,
  582.56103515625,
  'ORDEM DE SERVIÇO Nº 354, DE 28 DE NOVEMBRO DE 2018\nO SUBSECRETÁRIO DE ADMINISTRAÇÃO GERAL, DA SECRETARIA DE ESTADO DE\nPLANEJAMENTO, ORÇAMENTO E GESTÃO DO DISTRITO FEDERAL, Substituto, no uso de\nsuas atribuições regimentais e com fundamento no art. 67 da Lei nº 8.666, de 21 de junho de 1993,\ne no art. 41 do Decreto nº 32.598, de 15 de dezembro de 2010, e ainda, acatando as solicitações da\nárea competente, resolve:\nArt. 1º Retificar o item 7 do art. 1º da ORDEM DE SERVIÇO Nº 238, DE 11 DE SETEMBRO DE\n2018, publicada no DODF nº 173, de 11/09/2018. ONDE SE LÊ: RA-XII/Anexo Sede (JM e\nConselho), LEIA-SE: RA-XII Samambaia - Anexo da Administração (Junta Militar, Conselhos, Núcleo\nde Feira, Transporte e Gerências).\nArt. 2º Esta Ordem de Serviço entra em vigor na data de sua publicação.\nArt. 3º Revogam-se as disposições em contrário.\nNAUM ROSIVALDO DOS SANTOS',
  4,
  0),
 (56.69187545776367,
  592

In [5]:
_TRASH_EXPRESSIONS = [
    "SUMÁRIO",
    "DIÁRIO OFICIAL",
    "SEÇÃO (I|II|III)",
    "SEÇÃO",
]

_TRASH_COMPILED = re.compile('|'.join(_TRASH_EXPRESSIONS))


def is_bold(flags):
    return flags & 2 ** 4


def textBlock_topage(lis, page_width, pnum):
    """Given a text_block list (of tuples), return one with 
    two more values at the end, indicating `page number` and
    `page width`.
    """
    return [TextBlockTrans(*i[:-1], pnum, page_width) for i in lis]

def text_blocks_transform(text_blocks: List,
                          keep_page_width=True):
    lis = []
    for idx, tb in enumerate(text_blocks):
        p_num = tb[-2]
        p_num = tb.page
        
        
        p_width = tb[-1]
        p_width = tb.pwidth
        
        
        p_num *= 2
        x0, y0, x1, y1 = tb[:4]
        x0 = tb.x0
        
        p_num += int(x0 >( p_width / 2))
        if keep_page_width:            
            lis.append( TextBlockTrans( *(tb[:-2]), p_num, p_width ) )
        else:
            lis.append( TextBlockTrans( *(tb[:-2]), p_num, ) )
    return lis

def page_transform(blocks, keep_page_width=True, inplace=False):
    """Increases page numbers of blocks.
        This function takes an list of dictionaries each of wich
        having at least 'page', 'page_width' and 'bbox' as keys,
        and modify 'page' entry if bbox[0] > page_width / 2.
        Basically, "stacks" text based first on page and then if it is
        located on left/right half-horizontal.


    Args:
        blocks: List[Dict]
        keep_page_width: whether to drop or not `page_width`
            dict entries
    Returns:
        the modified list.
    WARNING:
        blocks is modified.
    """
    if not inplace:
        blocks = [i.copy() for i in blocks]
    for d in blocks:
        p_num = d['page']
        p_width = d.pop('page_width') if \
                    not keep_page_width else d['page_width']
        p_num *= 2
        x0, y0, x1, y1 = d['bbox']
        # Is top-left corner on left [horizontal] half of the page?
        p_num += int(x0 >( p_width / 2))
        # p_num = p_num + int(((x0 > p_width/2) and (x1 > p_width/2) ))
        # p_num = p_num + int(x0 > (p_width * .4) and x1 > (p_width / 2))
        d['page'] = p_num
    return blocks


def is_title_subtitle(span):
    return ((title_filter.BoldUpperCase.dict_text(span))
            and is_bold(span['flags'])
            and not re.search(_TRASH_COMPILED, span['text'])
            and 'calibri' not in span['font'].lower()
        )

def are_title_subtitle(span_list):
    return [is_title_subtitle(span) for span in span_list]

In [6]:
tb_paged = textBlock_topage(text_blocks, p_width, pnum)
# tb_paged[4:6];

In [7]:
tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
# tb_trans[5:7]
# [i.text for i in drop_header_footer(sort_transformed(
#     tb_trans
# ))]
tb_trans[0:1]

[TextBlockTrans
 	bbox: (56.666988372802734, 55.11705017089844, 765.406005859375, 70.56302642822266)
 	text: PÁGINA 2
 Nº 228, segunda-feira, 3 de dezembro de 2018
 D i ário Oficial do Distrito Federal
 	block_no: 0
 	page, pwidth: (2, None)]

## VERIFICAR SE UM BLOCO É CANDIDATO A TÍTULO

In [8]:
import re
UPPER_REG = re.compile(r'[A-Z]')
def is_upper(text: str):
    return text.upper() == text

In [9]:
def get_block_spans(block):
    span_lis = [] 
    for line in block['lines']: 
        for span in line['spans']: 
            span_lis.append(span) 
    return span_lis

titles_idx = [idx for (idx, bl) in enumerate(tb_trans)
          if is_upper(bl.text)]


In [10]:
block_spans = {}
for i in titles_idx:
    block_spans[i] = get_block_spans(extract[i])

In [11]:
block_spans.keys()

dict_keys([2, 3, 8, 9, 10, 11, 12, 15])

In [12]:
for k, span_list in block_spans.items():
    if all(are_title_subtitle(span_list)):
        print('\n'.join((sp['text'] for sp in span_list)))
        print('\t----- {} -----\n'.format(k))

SECRETARIA DE ESTADO DE PLANEJAMENTO,
ORÇAMENTO E GESTÃO
	----- 2 -----

SUBSECRETARIA DE ADMINISTRAÇÃO GERAL
	----- 3 -----

SECRETARIA DE ESTADO DA CASA CIVIL,
RELAÇÕES INSTITUCIONAIS E SOCIAIS
	----- 15 -----



In [13]:
def reading_sort_tuple(lis):
    return sorted(lis, key=lambda x: (x.page, int(x.y0), x.x0))

def reading_sort_dict(lis):
    return sorted(lis, key=lambda x: (x['page'], int(x['bbox'][1]), x['bbox'][0]))



# def sort_transformed(text_blocks_transformed: 
#     List[Union[TextBlockTrans, TextBlockPaged]]):
#     return sorted(text_blocks_transformed, key=trans_key)


def drop_dup_tbt(lis: List[TextBlockTrans]):
    """. This fun
    
    Sometimes, a span text apears multiple times, as if there exists
    multiple spans starting at the same point. Tihs function drops
    duplicate which matches this case.
    """
    dic = {}
    for tup in lis:
        dic[tuple([int(i) for i in tup[:2]])] = tup
    return list(dic.values())
ret = reading_sort_tuple(drop_dup_tbt(tb_trans))
print(ret[0])


def drop_header_footer(lis: List[tuple]):
    y0l = [ x.y0 for x in lis ]
    mi, ma = min(y0l), max(y0l)
    idx_mi, idx_ma = y0l.index(mi), y0l.index(ma)
    left, right = min(idx_mi, idx_ma), max(idx_mi, idx_ma)
    del lis[left]
    del lis[right-1]
    return lis



def mount_hierarchy(page: fitz.Page, pnum):
#     TODO : fazer para o documento inteiro
    p_width = page.MediaBox[2]
    
    text_blocks = page.getTextBlocks()
    extract = page.getTextPage().extractDICT()['blocks']

    tb_paged = textBlock_topage(text_blocks, p_width, pnum)    
    tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
    
    reading_sorted = reading_sort_tuple(drop_dup_tbt(tb_trans))
    
    cleaned_and_sorted = drop_header_footer(
        reading_sort_tuple(
        drop_dup_tbt(
            tb_trans
        )))

    hier = [('preambulo', [])]
    
    last_title = 'preambulo'

    for text_block in cleaned_and_sorted:
        spans = get_block_spans(extract[text_block.block_no])
        if all(are_title_subtitle(spans)):
            cpy = [i.copy() for i in spans]
            cpy = [(sp['text'], sp['bbox']) for sp in cpy]
            last_title = sorted(set(cpy),
                       key= lambda x:(x[1][1], x[1][0]))

            last_title = '\n'.join([i[0] for i in last_title])
            hier.append((last_title, []))
        else:
            hier[-1][1].append(text_block.text)
    return hier
            
            

TextBlockTrans
	bbox: (56.666988372802734, 55.11705017089844, 765.406005859375, 70.56302642822266)
	text: PÁGINA 2
Nº 228, segunda-feira, 3 de dezembro de 2018
D i ário Oficial do Distrito Federal
	block_no: 0
	page, pwidth: (2, None)


In [14]:
h=mount_hierarchy(p, pnum)
# h

In [61]:
from prextract.dodf_hierarchy import get_spans_by_page

def mount_doc_hierarchy(doc: fitz.Document):
    blocks_p0 = doc[0].getTextPage().extractDICT()['blocks']
    
    def get_first_title(blocks):
        sps = []
        for block in blocks_p0:
            for line in block['lines']:
                i = 0
                for sp in line['spans']:
                    sps.append(sp)
        idx=[i['text'] for i in sps].index("SEÇÃO I")
        return sps[idx+1]
                
    first_title = get_first_title(blocks_p0)
#     print('first title: ', first_title)
    
    TITLE_SIZE = first_title['size']
        
    prev_font_size = 0
    last_title = 'preambulo'
    hier = [ ([last_title], []) ]
    all_tbt = []
    
    _dbg = []
    prev_spans = []
    for idx, page in enumerate(doc):
        _dbg.append([])
        p_width = page.MediaBox[2]

        text_blocks = page.getTextBlocks()
        extract = page.getTextPage().extractDICT()['blocks']

        tb_paged = textBlock_topage(text_blocks, p_width, idx)    
        tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)

        cleaned_and_sorted = drop_header_footer(
            reading_sort_tuple(
            drop_dup_tbt(
                tb_trans
            )))
        for text_block in cleaned_and_sorted:
            spans = get_block_spans(extract[text_block.block_no])
#           FALTOU ORDENAR OS SPANS!!!


            for sp in spans:
                sp['page'] = idx
                sp['page_width'] = doc[idx].MediaBox[2]

            spans = reading_sort_dict(page_transform(spans))
            _dbg[-1].extend(spans)
            assert text_block[:4] == extract[text_block.block_no]['bbox']
            
            first_size = spans[0]['size']
            # Temos um título?
            cond1 = bool(spans)
            cond3 = first_size == TITLE_SIZE
            not_fake = [ not re.match(_TRASH_COMPILED, sp['text']) for sp in spans]            
            if  cond1 and all(are_title_subtitle(spans)) and cond3 and all(not_fake):
                # verificar se não estende o anterior (múltiplas linhas)
#                 raise "asdfjkl"
                if first_size == prev_font_size and hier[-1][0][0] != 'preambulo':
                    print("EXTENDING {} BY {}".format(hier[-1][0], text_block.text))
                    print("PREV_SPANS: ", prev_spans)
                    print("CURR_SPANS: ", spans)
                    print("---------------")
                    hier[-1][0].extend([text_block.text])
                else:
                    last = hier[-1]
                    hier[-1] = ('\n'.join(last[0]), last[1])
                    hier.append( ([text_block.text], []) )                
            else:  # Não é título/subtítulo em hipótese alguma
                hier[-1][1].append(text_block.text)
            if spans: prev_font_size = spans[0]['size']
            prev_spans = spans.copy()
#     return hier
    return _dbg

In [62]:
h = mount_doc_hierarchy(doc)

EXTENDING ['SECRETARIA DE ESTADO DE CULTURA'] BY SECRETARIA DE ESTADO DAS CIDADES
PREV_SPANS:  [{'size': 12.951430320739746, 'flags': 16, 'font': 'Arial-BoldMT', 'color': 2236191, 'text': 'SECRETARIA DE ESTADO DE CULTURA', 'bbox': (104.20772552490234, 323.2774353027344, 357.4254455566406, 337.7466125488281), 'page': 34, 'page_width': 814.9600219726562}]
CURR_SPANS:  [{'size': 12.951430320739746, 'flags': 16, 'font': 'Arial-BoldMT', 'color': 2236191, 'text': 'SECRETARIA DE ESTADO DAS CIDADES', 'bbox': (101.617431640625, 328.2587585449219, 360.01336669921875, 342.7279357910156), 'page': 34, 'page_width': 814.9600219726562}]
---------------
EXTENDING ['TRIBUNAL DE JUSTIÇA DO DISTRITO\nFEDERAL E DOS TERRITÓRIOS'] BY PODER EXECUTIVO
PREV_SPANS:  [{'size': 12.951430320739746, 'flags': 16, 'font': 'Arial-BoldMT', 'color': 2236191, 'text': 'SEÇÃO II', 'bbox': (201.1920166015625, 76.56258392333984, 259.4458923339844, 91.0317611694336), 'page': 42, 'page_width': 814.9600219726562}]
CURR_SPANS:  

In [55]:
from collections import Counter
# print(Counter([b.page for b in dbg[20] if b.page]))
for k in [i['text'] for i in  h[20]]:
    print(k)
    input(10*'-')

SEMA/SEGETH/NOVACAP/JBB/CEB. Portaria 06, de 01/03/2018 nomeia membros. 2 - GRUPO DE
----------
TRABALHO EM SUBSTITUIÇÃO À COMISSÃO PERMANENTE DO CADASTRO DISTRITAL
----------
DE ENTIDADES AMBIENTALISTAS, DE MORADORES E DE ENTIDADES PRIVADAS DE
----------
ENSINO SUPERIOR (CP-CEAMPES) especificamente para o primeiro processo eletivo. Criado pela
----------
Resolução 07, de 19/12/2017 (deliberado na 63ª RE 23/08/2017). Objetivo: tem por ?nalidade
----------
deliberar sobre o cadastramento, recadastramento e descadastramento de Organizações Ambientalistas,
----------
Associações de Moradores e de Entidades Privadas de Ensino Superior. Composição: Fórum das
----------
ONGs/ OAB/DF/UnB. Presidência: OAB/DF. Prazo de Validade: (até o preenchimento de todas as
----------
vagas vacantes destinadas à sociedade civil). 3 - GRUPO DE TRABALHO - GT, PARA ANALISAR
----------
A SOLICITAÇÃO DA EMPRESA STERICYCLE DA CEILÂNDIA/DF. Criado pela Resolução 08, de
----------
20/12/2017. Composição: SEMA/SINE

----------
disposto nos artigos 211, 212 e 217, parágrafo único da Lei Complementar nº 840, de 23 de dezembro
----------
de 2011, e considerando o que consta do Processo nº 150.003.075/2016, resolve:
----------
Art. 1º Prorrogar, por 60 (sessenta) dias, o prazo para conclusão dos trabalhos da Comissão de
----------
Processo Disciplinar, instaurada pela Portaria nº 307, de 27 de outubro de 2017, publicada no DODF
----------
nº 342, de 02 de outubro de 2018, pág. 23.
----------
Art. 2º Esta Portaria entra em vigor na data de sua publicação.
----------
LUIS GUILHERME ALMEIDA REIS
----------
TRIBUNAL DE JUSTIÇA DO DISTRITO
----------
FEDERAL E DOS TERRITÓRIOS
----------
S E C R E TA R I A DO CONSELHO ESPECIAL E DA M A G I S T R AT U R A
----------
AÇÃO DIRETA DE INCONSTITUCIONALIDADE
----------
PUBLICAÇÃO DE ACÓRDÃO
----------
Número Processo: 2015002024295-2ADI - (0024794-94.2015.8.07.0000 - Res. 65 CNJ); Acórdão :
----------
931429; Relator: HUMBERTO ULHÔA; Relator Designado: MARIO-ZAM B

KeyboardInterrupt: 

In [19]:
def get_page_blocks(page: fitz.Page):
    return page.getTextPage().extractDICT()['blocks']
p20_blocks = get_page_blocks(doc[20])
sps=[]
for lis in p20_blocks:
    sps.extend(get_block_spans(lis))

for sp in sps:
    sp['page'] = 20
    sp['page_width'] = doc[20].MediaBox[2]

sps = page_transform(sps)
ordered = reading_sort_dict(sps)

In [46]:
_dbg = mount_doc_hierarchy(doc)

first title:  {'size': 12.951430320739746, 'flags': 16, 'font': 'Arial-BoldMT', 'color': 2236191, 'text': 'PODER LEGISLATIVO', 'bbox': (162.99427795410156, 695.5653076171875, 298.59423828125, 710.0344848632812)}
EXTENDING ['SECRETARIA DE ESTADO DE CULTURA'] BY SECRETARIA DE ESTADO DAS CIDADES
EXTENDING ['TRIBUNAL DE JUSTIÇA DO DISTRITO\nFEDERAL E DOS TERRITÓRIOS'] BY PODER EXECUTIVO
EXTENDING ['TRIBUNAL DE CONTAS DO DISTRITO FEDERAL'] BY PODER LEGISLATIVO


In [48]:
_dbg[21]

[{'size': 7.970109939575195,
  'flags': 4,
  'font': 'TimesNewRomanPSMT',
  'color': 2236191,
  'text': 'Secretaria de Estado de Políticas para Crianças,',
  'bbox': (56.70298385620117,
   545.1553955078125,
   214.81883239746094,
   553.981689453125),
  'page': 0,
  'page_width': 814.9600219726562},
 {'size': 7.970109939575195,
  'flags': 4,
  'font': 'TimesNewRomanPSMT',
  'color': 2236191,
  'text': '50',
  'bbox': (336.8802795410156,
   550.8839111328125,
   344.8503723144531,
   559.710205078125),
  'page': 0,
  'page_width': 814.9600219726562},
 {'size': 7.970109939575195,
  'flags': 4,
  'font': 'TimesNewRomanPSMT',
  'color': 2236191,
  'text': '83',
  'bbox': (378.9275817871094,
   550.8839111328125,
   386.8976745605469,
   559.710205078125),
  'page': 0,
  'page_width': 814.9600219726562},
 {'size': 7.970109939575195,
  'flags': 4,
  'font': 'TimesNewRomanPSMT',
  'color': 2236191,
  'text': 'adolescentes e Juventude ..........................................................